In [ ]:
import numpy as np
np.seterr(invalid='ignore')

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 13})

from my_functions import *
from load_mocks import *

from scipy.stats import binned_statistic

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))
w_lya = 1215.67
filter_tags = load_filter_tags()

In [ ]:
from minijpas_LF_and_puricomp import load_mocks
pm_flx, pm_err, zspec, EW_lya, L_lya, is_qso, is_sf, is_gal, is_LAE, where_hiL =\
    load_mocks('train', 'minijpas', add_errs=True)

In [ ]:
N_sources = pm_flx.shape[1]
N_sources

In [ ]:
mag = flux_to_mag(pm_flx[-2], w_central[-2])
mag[np.isnan(mag)] = 99.

In [ ]:
ew0_cut = 15

# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(
    pm_flx, pm_err, IGM_T_correct=True)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0_cut)
lya_lines, lya_cont_lines, line_widths = identify_lines(
    line, pm_flx, cont_est_lya, first=True, return_line_width=True
)
lya_lines = np.array(lya_lines)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(
    pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
                           400, obs=True)
other_lines = identify_lines(line_other, cont_est_other, pm_err)

# Compute z
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] = z_NB(np.array(lya_cont_lines)
                                                  [np.where(np.array(lya_lines) != -1)])

nice_z = np.abs(z_Arr - zspec) < 0.16

%xdel cont_est_other
%xdel cont_err_other


In [ ]:
mag_min = 17
mag_max = 24

nb_min = 5
nb_max = 15
# nb_min = 16
# nb_max = 23

# Used later!!
L_min = 40
L_max = 50

nbs_to_consider = np.arange(nb_min, nb_max + 1)

nb_cut = (np.array(lya_lines) >= nb_min) & (np.array(lya_lines) <= nb_max)

z_min = (w_central[nb_min] - nb_fwhm_Arr[nb_min] * 0.5) / w_lya - 1
z_max = (w_central[nb_max] + nb_fwhm_Arr[nb_max] * 0.5) / w_lya - 1
print(f'z interval: ({z_min:0.2f}, {z_max:0.2f})')

z_cut = (z_min < z_Arr) & (z_Arr < z_max)
zspec_cut = (z_min < zspec) & (zspec < z_max)
ew_cut = EW_lya > ew0_cut
mag_cut = (mag > mag_min) & (mag < mag_max)

nice_lya = nice_lya_select(
    lya_lines, other_lines, pm_flx, pm_err, cont_est_lya, z_Arr
)

snr = np.empty(N_sources)
for src in range(N_sources):
    l = lya_lines[src]
    snr[src] = pm_flx[l, src] / pm_err[l, src]

nice_lya = (nice_lya & z_cut & mag_cut & (snr > 6))

In [ ]:
_, _, L_Arr_uncorr, L_e_Arr_alt, _, _ = EW_L_NB(
    pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_lines, N_nb=0
)

In [ ]:
def compute_L_Lbin_err(L_Arr, L_lya, L_binning):
    '''
    Computes the errors due to dispersion of L_retrieved with some L_retrieved binning
    '''
    L_Lbin_err_plus = np.ones(len(L_binning) - 1) * 99
    L_Lbin_err_minus = np.ones(len(L_binning) - 1) * 99
    median = np.ones(len(L_binning) - 1) * 99
    last = [99., 99.]
    for i in range(len(L_binning) - 1):
        in_bin = (10 ** L_Arr >= L_binning[i]) & (10 ** L_Arr < L_binning[i + 1])
        if count_true(in_bin) == 0:
            L_Lbin_err_plus[i] = last[0]
            L_Lbin_err_minus[i] = last[1]
            continue
        perc = np.nanpercentile((L_Arr - L_lya)[in_bin], [16, 50, 84])
        L_Lbin_err_plus[i] = perc[2] - perc[1]
        
        last = [L_Lbin_err_plus[i], L_Lbin_err_minus[i]]
        median[i] = perc[1]

    return L_Lbin_err_plus, median

L_binning = np.logspace(41, 46, 20 + 1)
L_bin_c = [L_binning[i : i + 1].sum() * 0.5 for i in range(len(L_binning) - 1)]
mask = (nice_z & nice_lya & (EW_lya > 0) & (L_lya > 43))
L_Lbin_err, median_L = compute_L_Lbin_err(
    L_Arr_uncorr[mask], L_lya[mask], L_binning
)
np.save('npy/L_nb_err.npy', L_Lbin_err)
np.save('npy/L_bias.npy', median_L)
np.save('npy/L_nb_err_binning.npy', L_binning)

In [ ]:
L_Lbin_err

In [ ]:
# Correct L_Arr with the median
mask_median_L = (median_L < 10)
L_Arr = L_Arr_uncorr - \
    np.interp(L_Arr_uncorr, np.log10(L_bin_c)[
              mask_median_L], median_L[mask_median_L])

In [ ]:
# Apply bin err
L_binning_position = binned_statistic(
        10 ** L_Arr, None, 'count', bins=L_binning
).binnumber
L_binning_position[L_binning_position > len(L_binning) - 2] = len(L_binning) - 2
L_e_Arr = L_Lbin_err[L_binning_position]

In [ ]:
def do_this(mmin, mmax):
    mmask = (mag > mmin) & (mag < mmax)

    fig, ax = plt.subplots(figsize=(7, 6))

    to_hist1 = ((L_lya - L_Arr) / L_e_Arr)[nice_lya & mmask]
    to_hist2 = ((10**L_lya - 10**L_Arr) / L_e_Arr_alt)[nice_lya & mmask]

    bins = np.linspace(-5, 5, 40)
    ax.hist(to_hist1, bins=bins, histtype='step', label='Dist err')
    ax.hist(to_hist2, bins=bins, histtype='step', label='Prop err')

    # for a in np.nanpercentile(to_hist1, [16, 50, 84]):
    #     ax.axvline(a)

    ax.legend(fontsize=20)

    plt.show()

do_this(17, 24)
do_this(23, 24)

In [ ]:
# %xdel pm_flx
# %xdel pm_err
# %xdel cont_est_lya
# %xdel cont_err_lya

In [ ]:
def contour_curve(xx, yy, bins_xx, bins_yy):
    Z, x, y = np.histogram2d(xx, yy, bins=(bins_xx, bins_yy))

    H_min = np.amin(Z)
    H_max = np.amax(Z)

    y_centers = 0.5 * (y[1:] + y[:-1])
    x_centers = 0.5 * (x[1:] + x[:-1])

    N_bins = 10000

    H_Arr = np.linspace(H_min, H_max, N_bins)[::-1]

    fact_up_Arr = np.zeros(N_bins)

    TOTAL_H = np.sum(Z)

    for iii in range(0, N_bins):

        mask = Z > H_Arr[iii]

        fact_up_Arr[iii] = np.sum(Z[mask]) / TOTAL_H

    H_value_68 = np.interp(0.683, fact_up_Arr, H_Arr)  # 1sigma
    H_value_95 = np.interp(0.954, fact_up_Arr, H_Arr)  # 2sigma
    # H_value_90 = np.interp(0.900, fact_up_Arr, H_Arr) # 2sigma
    H_value_99 = np.interp(0.997, fact_up_Arr, H_Arr) # 2sigma
    levels = [H_value_99, H_value_95, H_value_68]

    return x_centers, y_centers, Z, levels


In [ ]:
def plot_contours(is_qso, maskk, title):
    fig, ax = plt.subplots(figsize=(3.5, 3.5))

    bins_xx = np.linspace(42, 46, 30)
    bins_yy = np.linspace(42, 46, 30)

    # All LAEs
    # mask = (nice_lya & nice_z & maskk)
    # x_centers, y_centers, Z, levels = contour_curve(L_lya[mask], L_Arr[mask],
    #                                                 bins_xx, bins_yy)
    # ax.contour(x_centers, y_centers, Z.T, levels=levels, colors='k', zorder=99)
    # mask = (nice_lya & nice_z & maskk)
    # x_centers, y_centers, Z, levels = contour_curve(L_lya[mask], L_Arr_uncorr[mask],
    #                                                 bins_xx, bins_yy)
    # ax.contour(x_centers, y_centers, Z.T, levels=levels, colors='k')

    mask = (is_sf & nice_lya & nice_z & maskk)
    x_centers, y_centers, Z, levels = contour_curve(L_lya[mask], L_Arr[mask],
                                                    bins_xx, bins_yy)
    ax.contour(x_centers, y_centers, Z.T, levels=levels, colors='C1')
    x_centers, y_centers, Z, levels = contour_curve(L_lya[mask], L_Arr_uncorr[mask],
                                                    bins_xx, bins_yy)
    # ax.contour(x_centers, y_centers, Z.T, levels=levels,
    #            colors='C1', linestyles='--')

    mask = ((mag[is_qso] < 24) & nice_lya[is_qso]
            & nice_z[is_qso] & maskk[is_qso])
    x_centers, y_centers, Z, levels = contour_curve(L_lya[is_qso][mask], L_Arr[is_qso][mask],
                                                    bins_xx, bins_yy)
    ax.contour(x_centers, y_centers, Z.T, levels=levels, colors='C0')
    x_centers, y_centers, Z, levels = contour_curve(L_lya[is_qso][mask], L_Arr_uncorr[is_qso][mask],
                                                    bins_xx, bins_yy)
    # ax.contour(x_centers, y_centers, Z.T, levels=levels,
    #            colors='C0', linestyles='--')

    x = np.linspace(40, 48, 100)
    ax.plot(x, x, linestyle='--', color='red', label='1:1')

    # Lx = [(L_binning[i] + L_binning[i + 1]) / 2 for i in range(len(L_binning) - 1)]
    # ax.plot(Lx, Lx + median_L, c='k')
    # ax.plot(Lx, Lx + median_L - L_Lbin_err[1], c='gray')
    # ax.plot(Lx, Lx + median_L + L_Lbin_err[0], c='gray')

    # Plot dummy lines to make the legend
    ax.plot([-99, -98], [-99, -98], label='QSO LAEs', c='C0')
    ax.plot([-99, -98], [-99, -98], label='SF LAEs', c='C1')

    ax.set_ylabel(r'Retrieved $\log L_{\mathrm{Ly}\alpha}$', fontsize=15)
    ax.set_xlabel(r'Real $\log L_{\mathrm{Ly}\alpha}$', fontsize=15)

    ax.set_ylim((42, 46))
    ax.set_xlim((42, 46))

    ax.legend(fontsize=9)
    ax.set_title(title, fontsize=13)

    filename = f'figures/L_lya_contours_m{mmin}-{mmax}.pdf'
    plt.savefig(filename, bbox_inches='tight', facecolor='w', edgecolor='w')
    plt.show()

mmin = 17
mmax = 24
title = f'{mmin} < $r$ < {mmax}'
maskk = (mag > mmin) & (mag < mmax) & (EW_lya > 20)
plot_contours(is_qso, maskk, title)

In [ ]:
gal_area = 5.54
bad_qso_area = 200
sf_area = 200
good_qso_area = 400
hiL_qso_area = 4000

# The proportional factors are made in relation to bad_qso
# So bad_qso_factor = 1
gal_factor = bad_qso_area / gal_area
good_qso_factor = bad_qso_area / good_qso_area
hiL_factor = bad_qso_area / hiL_qso_area

In [ ]:
def do_this(mmin, mmax):
    nice = (nice_lya & nice_z) & (mag > mmin) & (mag < mmax)
    not_nice = (nice_lya & ~nice_z)
    z_cut = (z_Arr > z_min) & (z_Arr < z_max)

    good_qso_loL = L_Arr[z_cut & nice & is_qso & ~where_hiL]
    good_qso_hiL = L_Arr[z_cut & nice & is_qso & where_hiL]
    good_sf = L_Arr[z_cut & nice & is_sf]

    bad_qso = L_Arr[z_cut & not_nice & ~is_LAE]
    bad_qso_LAE_loL = L_Arr[z_cut & not_nice & is_qso & is_LAE & ~where_hiL]
    bad_qso_LAE_hiL = L_Arr[z_cut & not_nice & is_qso & is_LAE & where_hiL]
    bad_sf = L_Arr[z_cut & not_nice & is_sf]
    bad_gal = L_Arr[z_cut & nice_lya & is_gal & (zspec < 1.9)]

    bins = np.linspace(42.75, 46, 25)
    bin_c = [bins[i : i + 2].sum() * 0.5 for i in range(len(bins) - 1)]

    # All histograms are made for 1deg2

    h_good_qso_loL = np.histogram(good_qso_loL, bins)[0] / good_qso_area
    h_good_qso_hiL = np.histogram(good_qso_hiL, bins)[0] / hiL_qso_area
    h_good_sf = np.histogram(good_sf, bins)[0] / sf_area
    h_bad_qso = np.histogram(bad_qso, bins)[0] / bad_qso_area
    h_bad_qso_LAE_loL = np.histogram(bad_qso_LAE_loL, bins)[0] / good_qso_area
    h_bad_qso_LAE_hiL = np.histogram(bad_qso_LAE_hiL, bins)[0] / hiL_qso_area
    h_bad_sf = np.histogram(bad_sf, bins)[0] / sf_area
    h_bad_gal = np.histogram(bad_gal, bins)[0] / gal_area

    h_good_qso = h_good_qso_loL + h_good_qso_hiL
    h_bad_qso = h_bad_qso + h_bad_qso_LAE_hiL + h_bad_qso_LAE_loL

    print(h_good_qso.sum())

    fig, ax = plt.subplots(figsize=(7, 6))

    ax.plot(bin_c, h_good_qso,
            color='C0', drawstyle='steps', label='Good QSO')
    ax.plot(bin_c, h_good_sf,
            color='C1', drawstyle='steps', label='Good SF')
    ax.plot(bin_c, h_bad_qso,
            color='C0', drawstyle='steps', ls='--', label='Contaminant QSO')
    # ax.plot(bin_c, h_bad_sf,
    #         color='C1', drawstyle='steps', ls='--', label='Bad SF')
    ax.plot(bin_c, h_bad_gal,
            color='C2', drawstyle='steps', ls='--', label='Contaminant GAL')

#     ax.set_ylim(1, 10)
#     ax.set_yscale('log')
    ax.set_ylabel('deg$^{-1}$', fontsize=15)
    ax.set_xlabel(r'$L_{\mathrm{Ly}\alpha}$', fontsize=15)

    ax.legend(fontsize=12)
    ax.set_title(f'$r$ {mmin}-{mmax}', fontsize=20)

    plt.show()

do_this(17, 24)

In [ ]:
def do_this(mmin, mmax):
    nice = (nice_lya & nice_z) & (mag > mmin) & (mag < mmax)
    not_nice = (nice_lya & ~nice_z)
    z_cut = (z_Arr > z_min) & (z_Arr < z_max)

    good_qso_loL = mag[z_cut & nice & is_qso & ~where_hiL]
    good_qso_hiL = mag[z_cut & nice & is_qso & where_hiL]
    good_sf = mag[z_cut & nice & is_sf]

    bad_qso = mag[z_cut & not_nice & ~is_LAE]
    bad_qso_LAE_loL = mag[z_cut & not_nice & is_qso & is_LAE & ~where_hiL]
    bad_qso_LAE_hiL = mag[z_cut & not_nice & is_qso & is_LAE & where_hiL]
    bad_sf = mag[z_cut & not_nice & is_sf]
    bad_gal = mag[z_cut & nice_lya & is_gal & (zspec < 1.9)]

    bins = np.linspace(17, 25, 30)
    bin_c = [bins[i : i + 2].sum() * 0.5 for i in range(len(bins) - 1)]

    # All histograms are made for 1deg2

    h_good_qso_loL = np.histogram(good_qso_loL, bins)[0] / good_qso_area
    h_good_qso_hiL = np.histogram(good_qso_hiL, bins)[0] / hiL_qso_area
    h_good_sf = np.histogram(good_sf, bins)[0] / sf_area
    h_bad_qso = np.histogram(bad_qso, bins)[0] / bad_qso_area
    h_bad_qso_LAE_loL = np.histogram(bad_qso_LAE_loL, bins)[0] / good_qso_area
    h_bad_qso_LAE_hiL = np.histogram(bad_qso_LAE_hiL, bins)[0] / hiL_qso_area
    h_bad_sf = np.histogram(bad_sf, bins)[0] / sf_area
    h_bad_gal = np.histogram(bad_gal, bins)[0] / gal_area

    h_good_qso = h_good_qso_loL + h_good_qso_hiL
    h_bad_qso = h_bad_qso + h_bad_qso_LAE_hiL + h_bad_qso_LAE_loL

    print(h_good_qso.sum())

    fig, ax = plt.subplots(figsize=(7, 6))

    ax.plot(bin_c, h_good_qso,
            color='C0', drawstyle='steps', label='Good QSO')
    ax.plot(bin_c, h_good_sf,
            color='C1', drawstyle='steps', label='Good SF')
    ax.plot(bin_c, h_bad_qso,
            color='C0', drawstyle='steps', ls='--', label='Contaminant QSO')
    # ax.plot(bin_c, h_bad_sf,
    #         color='C1', drawstyle='steps', ls='--', label='Bad SF')
    ax.plot(bin_c, h_bad_gal,
            color='C2', drawstyle='steps', ls='--', label='Contaminant GAL')

#     ax.set_ylim(1, 10)
#     ax.set_yscale('log')
    ax.set_ylabel('deg$^{-1}$', fontsize=15)
    ax.set_xlabel(r'$r$ (magAB)', fontsize=15)

    ax.legend(fontsize=12)

    plt.show()

do_this(17, 24)